# Intro



In [4]:
# Activate Logging
import logging
#logging.basicConfig(level=logging.DEBUG)
logging.basicConfig(level=logging.INFO)

In [5]:
from stabledracor.client import StableDraCor

In [6]:
# check for GitHub Token (see FAQ Notebook)
import os
github_token = os.environ.get("GITHUB_TOKEN")
assert github_token is not None, "It is recommended to use a GitHub Personal Access Token. See FAQ Notebook for details."

In [7]:
dracor = StableDraCor(name="my-stable-dracor", github_access_token=github_token)

INFO:root:Initialized new StableDraCor instance: 'my-stable-dracor' (ID: ba657fe8-b3b2-4e68-a8f9-04c5c191b2b8).
INFO:root:Docker is available.
